# Import Dataset and Libraries

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./iris.csv')
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [2]:
df['variety'] = df['variety'].map({'Setosa': 0,
                                  'Versicolor': 1,
                                  'Virginica':2},
                                 na_action=None)
df['variety'].unique()

array([0, 1, 2])

In [3]:
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


## Some Preprocessing

To make life a little easier, I will be changing the species dataframe to only have two classes. With three classes this becomes a much more difficult algorithm to build from scratch as the output is non-binary and many applications will only need a binary application

In [4]:
df = df[df['variety'] < 2]
df.tail()

,sepal.length,sepal.width,petal.length,petal.width,variety
95,5.7,3.0,4.2,1.2,1
96,5.7,2.9,4.2,1.3,1
97,6.2,2.9,4.3,1.3,1
98,5.1,2.5,3.0,1.1,1
99,5.7,2.8,4.1,1.3,1


In [5]:
df['variety'].unique()

array([0, 1])

# Decision Tree from Scratch

First up is the TreeNode which is exactly what it says, a class for the nodes in the tree, whether they be decision or leaf nodes.

The `left` and `right` parameters are pointers to children. The `feature` is the feature (column) that the decision node will be splitting off of. The `threshold` value is the numeric splitting value according to the corresponding feature; and the `info_gain` parameter displays the information gain for each decision node.

In [24]:
class TreeNode():
    def __init__(self, left=None, right=None, feature=None, threshold=None, info_gain=None, value=None):
        self.threshold=threshold
        self.feature=feature
        self.right=right
        self.left=left
        self.info_gain=info_gain

        # Used for the leaf node
        self.value=value


The `DecisionTree()` class will hold all of the functions necessary to build the Decision Tree. The parameters are `max_depth` and `min_samples_split` which are both hyperparameters to keep the decsion tree from overfitting. The *max_depth* parameters keeps the tree from extending past a certain depth and the *min_samples_split* essentially says that a node must contain at least a certain number of samples in it to make the split. If the amount of sample in an internal node is less than the *min_samples_split*, then that node will become a leaf node.

In [32]:
class DecisionTree():
    def __init__(self, max_depth=None, min_samples_split=None):
        # Decision Tree hyperparameters to prevent from overfitting
        self.max_depth=max_depth
        self.min_samples_split=min_samples_split

        # initialize the root of the tree
        self.root=None        
        
    def dfs_build(self, df, depth=0):
        X = df.iloc[:, :-1].to_numpy()
        Y = df.iloc[:, -1].to_numpy()
        num_samples, _ = np.shape(X)
        
        if num_samples >= self.min_samples_split and depth <= self.max_depth:
            node_info = self.find_best_split(df)
            if node_info["info_gain"] > 0:
                left_subtree = self.dfs_build(node_info["left_df"], depth+1)
                right_subtree = self.dfs_build(node_info["right_df"], depth+1)
                return TreeNode(left_subtree, 
                               right_subtree, 
                               node_info["feature"], 
                               node_info["threshold"], 
                               node_info["info_gain"])
        #If there is no information gain
        else:
            leaf_value = self.calculate_leaf(Y)
            return TreeNode(value = leaf_value)
        
    def find_best_split(self, df):
        max_info_gain = -float("inf")
        TreeNode_stats = {}
        for col_index, feature in enumerate(df.columns):
            # We want to ignore the target column
            if col_index == len(df.columns) - 1:
                break
            # Within each feature find all the threshold values
            threshold_vals = df[feature].unique()
            for threshold in threshold_vals:
                # First create the "left" and "right" datasets
                left_df = df[df[feature] <= threshold]
                right_df = df[df[feature] > threshold]
                if left_df.empty or right_df.empty:
                    continue
                G = self.gain_from_gini(left_df.iloc[:,col_index].to_numpy(), 
                                        right_df.iloc[:, col_index].to_numpy(), 
                                        df.iloc[:,-1].to_numpy(), 
                                        threshold)
                if G > max_info_gain:
                    TreeNode_stats["info_gain"] = G
                    TreeNode_stats["feature"] = col_index      # CHANGED THIS WHEN DOING PREDICT FUNCTION
                    TreeNode_stats["threshold"] = threshold
                    TreeNode_stats["left_df"] = left_df
                    TreeNode_stats["right_df"] = right_df
        
        return TreeNode_stats
    
    def gain_from_gini(self, left_data, right_data, labels, threshold):

        samples = len(left_data) + len(right_data)
        
        weight_left = len(left_data) / samples
        weight_right = len(right_data) / samples
        
        parent_gini = self.get_gini_score(labels)
        leftchild_gini = self.get_gini_score(left_data)
        rightchild_gini = self.get_gini_score(right_data)
        
        gain = parent_gini - ((weight_left * leftchild_gini) + (weight_right * rightchild_gini))
        return gain
        
    def get_gini_score(self, Y):
        class_labels = np.unique(Y)
        gini_score = 0
        for label in class_labels:
            label_prob = len(Y[Y == label]) / len(Y)
            gini_score +=label_prob ** 2
        return 1 - gini_score
    
    def leafnode_value(self, labels):
        # find the majority label and return that
        counts = np.bincount(labels)
        return np.argmax(counts)
    
    def fit(self, dataframe):
        self.root = self.dfs_build(dataframe)
        
    def predict(self, test_set):
        test = test_set.to_numpy()
        results = []
        #2-D array now
        for row in test:
            ptr = self.root
            while ptr.value is None:
                index = ptr.feature
                if row[index] <= ptr.threshold:
                    ptr = ptr.left
                else:
                    ptr = ptr.right
            results.append(ptr.value)
        return results

# Shuffle and Split the Dataset

In [10]:
shuffled = df.sample(frac=1)
shuffled.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
94,5.6,2.7,4.2,1.3,1
29,4.7,3.2,1.6,0.2,0
27,5.2,3.5,1.5,0.2,0
61,5.9,3.0,4.2,1.5,1
74,6.4,2.9,4.3,1.3,1


In [28]:
length, _ = shuffled.shape
part = int(length * 0.2)
test_set = shuffled.iloc[:part, :]
train_set = shuffled.iloc[part:, :]

In [33]:
classifier = DecisionTree(max_depth=3, min_samples_split=3)
classifier.fit(train_set)
#     def __init__(self, max_depth=None, min_samples_split=None):